# Predicción de Abandono de Clientes en Beta Bank

# Contenido <a id='back'></a>

* [Introducción](#intro)
* [Objetivos](#obj)
* [Etapa 1. Descripción de los datos](#data_review)
* [Etapa 2. Preprocesamiento de datos](#data_preprocessing)
* [Etapa 3. Desarrollo de modelos](#modelo)
* [Conclusiones](#end)

## Introducción <a id='intro'></a>
Beta Bank, una institución financiera, ha estado experimentando una pérdida constante de clientes mes a mes. Los banqueros han identificado que retener a los clientes existentes es significativamente más económico que adquirir nuevos. Con el fin de mitigar esta pérdida, Beta Bank ha decidido desarrollar un modelo de machine learning para predecir si un cliente dejará el banco en el futuro cercano.

## Objetivos: <a id='obj'></a>
Crear un modelo de machine learning que prediga con precisión el abandono de clientes en Beta Bank. El modelo deberá alcanzar un valor F1 mínimo de 0.59. Además, se evaluará la métrica ROC-AUC para proporcionar una visión más completa del rendimiento del modelo.

## Etapas:
El proyecto consistirá en tres etapas:
 1. Descripción de los datos.
 2. Preprocesamiento de datos.
 3. Desarrollo de modelos.


## Etapa 1. Descripción de los datos <a id='data_review'></a>

El conjunto de datos proporcionado contiene información sobre el comportamiento pasado de los clientes de Beta Bank, incluyendo características demográficas, financieras y de comportamiento. Las principales características del conjunto de datos son:

RowNumber: Índice del conjunto de datos.

CustomerId: Identificador único del cliente.

Surname: Apellido del cliente.

CreditScore: Puntaje de crédito del cliente.

Geography: País de residencia del cliente.

Gender: Género del cliente.

Age: Edad del cliente.

Tenure: Duración de la relación del cliente con el banco (en años).

Balance: Saldo de la cuenta del cliente.

NumOfProducts: Número de productos bancarios utilizados por el cliente.

HasCrCard: Indicador de si el cliente tiene una tarjeta de crédito (1 - sí, 0 - no).

IsActiveMember: Indicador de si el cliente es un miembro activo (1 - sí, 0 - no).

EstimatedSalary: Salario estimado del cliente.

Exited: Indicador de si el cliente ha dejado el banco (1 - sí, 0 - no).

## Etapa 2. Preprocesamiento de datos <a id='data_preprocessing)'>

In [29]:
#importamos librerias necesarias para el proyecto
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

## Exploracion Inicial de Datos

In [3]:
data = pd.read_csv("Churn.csv")
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB
None


Observamos que la columna Tenure tiene valores nulos, tambien podemos diferenciar las categorias mediante el tipo para su posterior analisis. Escogeremos la mediana para completar los valores nulos. Adicionalmente agregaremos dummys para las variables categoricas, que serian Geography y Gender. Tambien eliminamos la columna object que seria surname ya que no aporta al modelo.

In [4]:
#rellenamos los valores nulos con la mediana y eliminamos el tipo object
data["Tenure"].fillna(data["Tenure"].median(), inplace=True)
data = data.drop(['Surname','RowNumber','CustomerId'], axis=1)

#creamos dummys a las variables categoricas
data = pd.get_dummies(data, columns=["Geography","Gender"], drop_first=True)

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CreditScore        10000 non-null  int64  
 1   Age                10000 non-null  int64  
 2   Tenure             10000 non-null  float64
 3   Balance            10000 non-null  float64
 4   NumOfProducts      10000 non-null  int64  
 5   HasCrCard          10000 non-null  int64  
 6   IsActiveMember     10000 non-null  int64  
 7   EstimatedSalary    10000 non-null  float64
 8   Exited             10000 non-null  int64  
 9   Geography_Germany  10000 non-null  bool   
 10  Geography_Spain    10000 non-null  bool   
 11  Gender_Male        10000 non-null  bool   
dtypes: bool(3), float64(3), int64(6)
memory usage: 732.6 KB


## Analisis de Datos

In [5]:
class_counts = data['Exited'].value_counts()
class_proportions = class_counts / len(data)

print(class_counts)
print(class_proportions)

Exited
0    7963
1    2037
Name: count, dtype: int64
Exited
0    0.7963
1    0.2037
Name: count, dtype: float64


## Etapa 3. Desarrollo de modelos <a id='modelo'></a>

In [34]:
features = data.drop(['Exited'], axis=1)
target = data['Exited']

#separamos el dataset en 60% entrenamiento, 20% validacion y el resto en prueba

features_train, features_remaining, target_train, target_remaining = train_test_split(features, target, test_size=0.4,random_state=12345)
features_valid, features_test, target_valid, target_test = train_test_split(features_remaining, target_remaining, test_size=0.5,random_state=12345)

#entrenamos el primer modelo  
model = LogisticRegression(random_state=12345,solver="liblinear", class_weight='balanced')
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)

print(f'F1 = {f1_score(target_valid, predicted_valid)}, ROC = {roc_auc_score(target_valid, model.predict_proba(features_valid)[:, 1])}')
print(f'Acurracy score = {accuracy_score(target_valid, predicted_valid)}')


F1 = 0.4970711297071129, ROC = 0.7582673497904052
Acurracy score = 0.6995


Como nos salio el F1 por debajo de la metrica a pesar de haber equilibrado los pesos, debemos intentar probar otro modelo. Utilizando otro modelo de ML como el RandomForest

In [33]:
#entrenamos un segundo modelo
model_tree = RandomForestClassifier(random_state=12345, class_weight='balanced')
model_tree.fit(features_train, target_train)
predicted_valid_tree = model_tree.predict(features_valid)

print(f'F1 = {f1_score(target_valid, predicted_valid_tree)}, ROC = {roc_auc_score(target_valid, model_tree.predict_proba(features_valid)[:, 1])}')
print(f'Acurracy score = {accuracy_score(target_valid, predicted_valid_tree)}')

F1 = 0.5718701700154559, ROC = 0.8382188375201882
Acurracy score = 0.8615


Comprobamos que el modelo tree esta funcionando mejor que el modelo de regresion logistica, ahora ajustaremos los hiperparametros para seguir mejorando el modelo.

In [11]:
#ajustamos hiperparametros
n_estimators_values = [100, 200, 300]
max_depth_values = [10, 20, 30]

best_f1 = 0
best_roc_auc = 0
best_params = {}

for n_estimators in n_estimators_values:
    for max_depth in max_depth_values:
        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=12345, class_weight='balanced')
    
        model.fit(features_train, target_train)
        predicted_valid = model.predict(features_valid)

        f1 = f1_score(target_valid, predicted_valid)
        roc_auc = roc_auc_score(target_valid, model.predict_proba(features_valid)[:, 1])
    
        print(f'n_estimators: {n_estimators}, max_depth: {max_depth}, F1: {f1}, ROC: {roc_auc}')
        
        if f1 > best_f1:
            best_f1 = f1
            best_roc_auc = roc_auc
            best_params = {'n_estimators': n_estimators, 'max_depth': max_depth}

print(f'Mejor F1: {best_f1}, Mejor ROC-AUC: {best_roc_auc}, con hiperparámetros: {best_params}')

n_estimators: 100, max_depth: 10, F1: 0.6159420289855072, ROC: 0.8525320743532202
n_estimators: 100, max_depth: 20, F1: 0.5538461538461539, ROC: 0.8385583326780347
n_estimators: 100, max_depth: 30, F1: 0.5718701700154559, ROC: 0.8382188375201882
n_estimators: 200, max_depth: 10, F1: 0.6161251504211794, ROC: 0.8545418251985554
n_estimators: 200, max_depth: 20, F1: 0.5564142194744978, ROC: 0.8426209328631312
n_estimators: 200, max_depth: 30, F1: 0.56793893129771, ROC: 0.8408932125164077
n_estimators: 300, max_depth: 10, F1: 0.6148325358851675, ROC: 0.8548881253818376
n_estimators: 300, max_depth: 20, F1: 0.5766871165644172, ROC: 0.8436439550202941
n_estimators: 300, max_depth: 30, F1: 0.5788667687595712, ROC: 0.842709398193795
Mejor F1: 0.6161251504211794, Mejor ROC-AUC: 0.8545418251985554, con hiperparámetros: {'n_estimators': 200, 'max_depth': 10}


In [37]:
#aplicando tecnica de sobremuestreo al modelo 2
def upsample (features, target, repeat):
    features_zeros = features_train[target_train == 0]
    features_ones = features_train[target_train == 1]
    target_zeros = target_train[target_train == 0]
    target_ones = target_train[target_train == 1]

    arg1 = pd.concat([features_zeros]+[features_ones]*repeat)
    arg2 = pd.concat([target_zeros]+[target_ones]*repeat)

    features_upsampled, target_upsampled = shuffle(arg1, arg2, random_state=12345)

    return features_upsampled, target_upsampled

features_upsampled, target_upsampled = upsample(features_train, target_train, 10)

#entrenamos el modelo con el sobremuestreo

model_tree_upsampled = RandomForestClassifier(random_state=12345)
model_tree_upsampled.fit(features_upsampled, target_upsampled)
predicted_valid_tree_upsampled = model_tree_upsampled.predict(features_valid)

print(f'F1 = {f1_score(target_valid, predicted_valid_tree_upsampled)}, ROC = {roc_auc_score(target_valid, model_tree_upsampled.predict_proba(features_valid)[:, 1])}')
print(f'Acurracy score = {accuracy_score(target_valid, predicted_valid_tree_upsampled)}')

F1 = 0.5880758807588076, ROC = 0.8435585141453796
Acurracy score = 0.848


Con la tecnica del sobremuestreo, se obtiene un F1 menor por el cual concluimos que el metodo que utilizaremos para la prueba final del mejor modelo seria el class_weight ='balanced'.

In [35]:
#prueba final con el conjunto de prueba
model_final = RandomForestClassifier(n_estimators=200, max_depth=10, random_state=12345, class_weight='balanced')
model_final.fit(features_train, target_train)
predicted_valid_final = model.predict(features_test)

print(f'F1 = {f1_score(target_valid, predicted_valid_final)}, ROC = {roc_auc_score(target_valid, model_final.predict_proba(features_valid)[:, 1])}')
print(f'Acurracy score = {accuracy_score(target_test, predicted_valid_final)}')

F1 = 0.27081581160639195, ROC = 0.8545418251985554
Acurracy score = 0.686


## Conclusion <a id='end'></a>

En este proyecto, se desarrolló un modelo de machine learning para predecir el abandono de clientes en Beta Bank. 

El objetivo fue crear un modelo que alcanzara un valor F1 mínimo de 0.59 y una alta métrica ROC-AUC.

El modelo final, utilizando RandomForestClassifier con n_estimators = 200 y max_depth = 10, alcanzó un valor F1 de 0.616 y un ROC-AUC de 0.854.

Estos resultados superan el umbral mínimo requerido para el valor F1.